# Informed search - the A* algorithm

Depth-first search and breadth-first search considered in the previous lesson are completely *blind* algorithms: they're only concerned whether the currently considered state is a goal state or not. They're unable to distinguish whether a state is easy or hard to reach, or whether it is near or far of the goal. This makes them very inefficient search algorithms. To allievate the issue, we introduce informed search algorithms. The information is given to an algorithm in two ways:

1. By using an *action cost* function $c(s,a)$, which, given a state $s$ and an action $a$ available in this state, returns its cost as a non-negative number.
2. By using a *heuristic* $h(s)$, which, given a state, estimates the lowest cost to reach a goal state from the given state.

Given a sequence of actions $a_1, \ldots, a_n$ and an initial state $s_1$, we can express the *total cost* of reaching the state $s_{n+1}$ by executing the sequence as:
$$ c(s_1, a_1, \ldots, a_{n-1}) = \sum_{i=1}^n c(s_i, a_i) $$
and the *expected cost* of the solution as the sum of the total cost and the estimate cost of reaching the goal from the state $s_{n+1}$
$$ f(s_1, a_1, \ldots, a_n) = c(s_1, a_1, \ldots, a_n) + h(s_{n+1}) $$

The heuristic function is a bit tricky, because we want it to have two properties:
* *Admissibility*: It must never *overestimate* the true cost of reaching the goal. 
* *Consistency*: Let $s$ be a state such that $a$ is an available action in this state and $s'$ is the state reached by executing this action. The heuristic should fulfil triangle inequality, that is, the estimated cost to reach the goal from $s$ should be no greater than the cost of executing the action $a$ + the estimated cost of reaching the goal from the new state.
$$ h(s) \leq c(s, a) + h(s') $$

One can prove that admissibility follows from consistency, but consistency is important only if there are multiple paths to reach the same state (i.e., we are searching in a graph, not in a tree). Otherwise, admissability is sufficient.

Lets extend the class `Problem` from the previous lesson with two new functions `action_cost` and `heuristic`, which correspond to the functions $c(s,a)$ and $h(s)$ described above.

In [1]:
class Problem:
    @property
    def initial_state(self):
        ...
        
    def available_actions(self, state):
        ...        
        
    def do_action(self, state, action):
        ...
        return new_state
    
    def is_goal(self, state) -> bool:
        ...
        
    def action_cost(self, state, action) -> float:
        ...
        
    def heuristic(self, state) -> float:
        ...

To make a concrete example, lets revisit the vacuum world. 

![](aima-images/fig2_2.png)

Below, we assume a very simple model:
* Any action costs 1. This corresponds to searching for the shortest plan.
* The heuristic estimation is the number of fields which are still dirty. 


Lets consider the properties of the heuristic:
* Is is admissible? The heuristic value is equal to the number of 'Suck' actions that are yet to be executed and ignores the spatial aspect (i.e., moving between the rooms), thus never overestimating.
* Is it consistent? As a consequence of a single action the heuristic value can decrease by at most 1 (if the action happens to be 'Suck' and the room is dirty). The cost of any action is 1, so rewriting the triangle inequality we arrive at:
$$ h(s) \leq c(s, a) + h(s') = \begin{cases} 1 + (h(s)-1) & a=\text{'Suck' and the room was dirty} \\ 1 + h(s) & \text{otherwise} \end{cases} $$
* Is it the best we could have? By no means! We could include the spatial aspect.

In [2]:
class VacuumProblem(Problem):
    @property
    def initial_state(self):
        return (0, (True, True))
    
    def available_actions(self, state):
        return ["Left", "Suck", "Right"]
        
    def do_action(self, state, action):
        robot, dirty = state
        if action == "Left":
            return (max(robot-1, 0), dirty)
        elif action == "Suck":
            new_dirty = list(dirty)
            new_dirty[robot] = False
            return (robot, tuple(new_dirty))
        elif action == "Right":
            return (min(robot+1, len(dirty)-1), dirty)        
        raise Exception('Invalid action')
    
    def is_goal(self, state) -> bool:
        return not any(state[1])
    
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return sum(state[1])

## Task 1: Implement the A* algorithm

To implement the A* algorithm you must have a priority queue. Luckily, Python comes with one, so you don't need to implement it by yourself. Then, the algorithm is very simple: 
1. Start with a queue containing a single item - the initial state
2. Repeat until the queue is not empty:
  1. Pick an item with the lowest expected cost
  2. If this is the goal, return the sequence of actions necessary to reach this state
  3. Otherwise, for each available action, create a new entry in the queue corresponding to the state reached after executing the action.
  
Guard the algorithm against infinite loops: if you already visited a state, you don't need to visit it again (if your heuristic is consistent).

In the cell below implement the algorithm in a similar manner as the BFS and DFS in the previous lesson: the sole argument is an object of the class Problem and the function should return a list of actions to achieve a goal state from the initial state.
If it is impossible to reach the goal, return `None`.
Count the number of states visited during the search and print in out before returning from the function, it will be useful later on to compare different heuristics.

In [3]:
from queue import PriorityQueue

In [4]:
def astar(problem: Problem):
    queue = PriorityQueue()
    current_state = problem.initial_state
    path = ()
    cumulative_costs = 0
    visited_states_with_dist_from_start = {current_state:0}
    expected_distance_from_goal = problem.heuristic(current_state) + cumulative_costs
    queue.put((expected_distance_from_goal, current_state, path, cumulative_costs))
    count = 0
    
    while queue:
        expected_distance_from_goal, current_state, path, cumulative_costs = queue.get()
#         print(expected_distance_from_goal, cumulative_costs, path)
        if problem.is_goal(current_state):
#             print(len(visited_states_with_dist_from_start))
#             print(count)
            return path
        
        for action in problem.available_actions(current_state):
            count += 1
            next_state = problem.do_action(current_state, action)
            new_path = list(path)
            new_path.append(action)
            new_cumulative_costs = cumulative_costs + problem.action_cost(current_state, action)
            expected_distance_from_goal = problem.heuristic(next_state) + cumulative_costs
#             print("\t", expected_distance_from_goal, new_cumulative_costs, new_path)

            if next_state not in visited_states_with_dist_from_start.keys():
                visited_states_with_dist_from_start.update({next_state:expected_distance_from_goal})
            
            elif expected_distance_from_goal < visited_states_with_dist_from_start[next_state]:
                visited_states_with_dist_from_start[next_state] = expected_distance_from_goal
            
            else:
                continue
            
            queue.put((expected_distance_from_goal, next_state, tuple(new_path), new_cumulative_costs))
                
    
    print("there is no solution for this problem")
    return

Now lets test your code in the vacuum world!

In [5]:
astar(VacuumProblem())

('Suck', 'Right', 'Suck')

## Task 2: Variants of the vacuum world

Now lets consider a different take on the vacuum world in which the heuristic is not admissible and increases as the number of dirty fields decreases.

In [6]:
class VacuumProblem1(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return len(state[1]) - sum(state[1])
    
astar(VacuumProblem1())

('Suck', 'Right', 'Suck')

And another in which heuristic grossly overestimates the cost.

In [7]:
class VacuumProblem2(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return 10 * sum(state[1])
    
astar(VacuumProblem2())

('Suck', 'Right', 'Suck')

**Which of the three heuristic functions (`VacuumProblem`, `VacuumProblem1`, `VacuumProblem2`) is the best? Is it the expected answer given the properties of the heuristics? If not, explain why an unorthodox approach works better.**

The best one is the first one. The reason is that it is admissible and allows to perform low number of iterations needed to find a solution. The 3rd heuristic seems to be as good as the 1st one but it is because of little search space. The 3rd function is not admissible which means that A* combined with this heuristic does not guarantee the optimal solution which can be a drawback in some cases.

## Task 3: 8-puzzle problem

Recall the 8-puzzle problem from the previous lesson. Reuse your code and implement an extended version assuming that each action costs 1. Propose 3 (at least) admissible heuristics. This time don't change the initial state, your solution should be capable enough to solve this.

![](aima-images/fig3_4.png)

In [8]:
import random
from math import sqrt

In [9]:
class PuzzleProblem(Problem):
    @property
    def initial_state(self):
        """1st component of state:
        - the (row, col) of where the blank tile (the zero tile) is
        2nd component:
        -3x3 matrix representing the board and the arrangement of tiles"""
        return (4,(7,2,4,5,0,6,8,3,1))
    
    def generate_feasible_initial_state(self):
        state = (0, tuple(range(len(self.initial_state[1]))))
        for _ in range(70):
            action = random.choice(self.available_actions(state))
            state = self.do_action(state, action)
#         self.initial_state = state
        return state
        
    def available_actions(self, state):
        index_of_blank, board = state
        possible_moves = []
        dimension = int(sqrt(len(self.initial_state[1])))
        
        if index_of_blank//dimension - 1 >= 0:
            possible_moves.append("Up")
        if index_of_blank//dimension + 1 <= dimension - 1:
            possible_moves.append("Down")
        if index_of_blank%dimension - 1 >= 0:
            possible_moves.append("Left")
        if index_of_blank%dimension + 1 <= dimension - 1:
            possible_moves.append("Right")
        return possible_moves
        
    def do_action(self, state, action):
        dimension = int(sqrt(len(self.initial_state[1])))
        actions = {'Up': -dimension,
                  'Down': dimension,
                  'Right': 1,
                  'Left': -1}
        index_of_blank, board = state
        new_index_of_blank = index_of_blank + actions[action]
        new_board = list(board)
        new_board[index_of_blank], new_board[new_index_of_blank] = new_board[new_index_of_blank], new_board[index_of_blank]
        
        new_state = (new_index_of_blank, tuple(new_board))
        
        return new_state
    
    def is_goal(self, state) -> bool:
        return state == (0, tuple(range(len(state[1]))))
        
    def action_cost(self, state, action) -> float:
        return 1.0
        
    def heuristic(self, state) -> float:
#         number of misplaced tiles
        index, board = state
        
        num_of_misplaces_tiles = -1
        for index in range(len(board)):
            if index != board[index]:
                num_of_misplaces_tiles+=1
        return float(num_of_misplaces_tiles)

**Prove that this heuristic is admissible.**

This heuristic counts the number of misplaced tiles and substracts one. This heuristic is admissible because:
1. Every action as long as it is not the final one, can decrease number of misplaced tiles by one.
2. If there is just a one pair of misplaced tiles the heuristic returns 1 because of the substraction.
3. If there would always exist a sequence of actions which can place one tile on a proper position per one action, the heuristic would be still admissible and because of the substraction it would not overestimate number of steps in the final stage when just two tiles are misplaced.

In [10]:
class PuzzleProblem1(PuzzleProblem):
    def heuristic(self, state) -> float:     
        return 0

**Prove that this heuristic is admissible.**

0 is trivially fulfilling the admissibility property.

In [11]:
class PuzzleProblem2(PuzzleProblem):
    def heuristic(self, state) -> float:
#         Manhattan distance
        index, board = state
        manhattan_dist = 0
        dimension = int(sqrt(len(self.initial_state[1])))
        for idx, tile in enumerate(board):
            manhattan_dist += sum( (abs(idx//dimension - tile//dimension), #row of a current spot - row of a goal spot
                                   abs(idx%dimension - tile%dimension)) )  #col of a current spot - col of a goal spot
        return manhattan_dist/2

**Prove that this heuristic is admissible.**

Manhattan distance divided by 2 is admissible because:
1. One action is always involves two tiles which means that an action can decrease the cumulative distance by at moxt two.
2. If every move can decrease cumulative distance by 2 then the cumulative manhattan distance returns twice the expected number of actions needed to solve a puzzle times 2.
3. If the cumulative manhattan distance will be divided then it won't overestimate in situations when every move pushes both tiles towards their goals, which is not probable in reality.

Run your heuristics on the given problem.

In [12]:
problem = PuzzleProblem()

%timeit plan0 = astar(PuzzleProblem())
plan0 = astar(PuzzleProblem())
%time plan1 = astar(PuzzleProblem1())
plan1 = astar(PuzzleProblem1())
%timeit plan2 = astar(PuzzleProblem2())
plan2 = astar(PuzzleProblem2())

for plan_x in (plan0, plan1, plan2):
    plan = list(plan_x)
    state = problem.initial_state
    while not problem.is_goal(state) and plan:
        action = plan.pop(0)
        state = problem.do_action(state, action)
    
    print("\nReached state:", state)
    print("Plan lenght: ", len(plan_x))
    print("Is it goal?", problem.is_goal(state), '\n')
    
# print(plan0)
# print(plan1)
# print(plan2)
print("Is plan0==plan1?", plan0 == plan1)
print("Is plan0==plan2?", plan0 == plan2)
print("Is plan1==plan2?", plan1 == plan2)

701 ms ± 39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
CPU times: user 2.3 s, sys: 13.4 ms, total: 2.31 s
Wall time: 2.31 s
610 ms ± 1.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Reached state: (0, (0, 1, 2, 3, 4, 5, 6, 7, 8))
Plan lenght:  26
Is it goal? True 


Reached state: (0, (0, 1, 2, 3, 4, 5, 6, 7, 8))
Plan lenght:  26
Is it goal? True 


Reached state: (0, (0, 1, 2, 3, 4, 5, 6, 7, 8))
Plan lenght:  26
Is it goal? True 

Is plan0==plan1? True
Is plan0==plan2? False
Is plan1==plan2? False


**Which of the heuristics is the best for this task? Why is that?**

I think that the Manhattan distance works the best because of the lowest average time amoung other heuristics and the fact that finds the optimal lenght solution.

------------
The pictures and the description of 8-puzzle are from "Artificial Intelligence: A Modern Approach" 3rd ed.

In [13]:
seq = list(range(16))
random.shuffle(seq)
print((seq.index(0), tuple(seq)))

(0, (0, 9, 13, 11, 3, 1, 8, 7, 4, 14, 15, 6, 2, 12, 10, 5))


In [14]:
class HardcorePuzzleProblem1(PuzzleProblem1):
    @property
    def initial_state(self):
        return (5, (1, 5, 2, 3, 8, 0, 4, 7, 14, 9, 6, 10, 12, 13, 15, 11))

In [15]:
problem = HardcorePuzzleProblem1()
print(problem.generate_feasible_initial_state())

(7, (8, 2, 1, 3, 9, 4, 6, 0, 5, 10, 11, 7, 12, 13, 14, 15))


In [16]:
problem = HardcorePuzzleProblem1()
%time plan = astar(problem)
# plan = astar(problem)

plan_x = list(plan)
state = problem.initial_state
while not problem.is_goal(state) and plan_x:
    action = plan_x.pop(0)
    state = problem.do_action(state, action)
    
print("\nReached state:", state)
print("Plan lenght: ", len(plan))
print("Is it goal?", problem.is_goal(state), '\n')

CPU times: user 14.3 s, sys: 283 ms, total: 14.6 s
Wall time: 14.6 s

Reached state: (0, (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15))
Plan lenght:  18
Is it goal? True 

